In [1]:
import enviroment
import numpy as np
from tensorflow import keras

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-10-21 16:26:42.713286: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 16:26:42.733623: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 16:26:42.771245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 16:26:42.839886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 16:26:42.857821: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 16:26:42.968381: I tensorflow/core/platform/cpu_feature_gu

In [2]:
#Funzioni necessarie per TabQFunction
def state_to_index(state_vector):
    pericoli = state_vector[0] * 4 + state_vector[1] * 2 + state_vector[2]
    direzione_corrente = state_vector[3] * 3 + state_vector[4] * 2 + state_vector[5] * 1
    posizione_frutto = np.mod(state_vector[7] * 6 + state_vector[8] * 3 + state_vector[9] * 2 + state_vector[10], 8)
    return pericoli * 32 + direzione_corrente * 8 + posizione_frutto
def initialize_QTable():
    return np.zeros((256, 3))

In [18]:
def tab_function(file_path, N_GAMES, visual=False, speed=0):
    Q_table = np.load(file_path)
    env_visual = enviroment.LinearStateSnakeGame(visual=visual, speed=speed)
    score_result = []
    for _ in range(N_GAMES):
        state = env_visual.reset()
        game_over = False
        while not game_over:
            state = env_visual.get_state()
            row_Q_value = state_to_index(state)
            action = np.argmax(Q_table[row_Q_value])
            final_move = [0, 0, 0]
            final_move[action] = 1
            _, _, game_over, score = env_visual.play_step(final_move)
        score_result.append(score)  
    return score_result

def dqn_easy_state(file_path, N_GAMES, visual=False, speed=0):
    model = keras.models.load_model(file_path)
    env_visual = enviroment.LinearStateSnakeGame(visual=visual, speed=speed)
    score_result = []
    for _ in range(N_GAMES):
        state = env_visual.reset()
        state = env_visual.get_state()
        game_over = False
        while not game_over:
            action = np.argmax(model(state[np.newaxis])[0])
            final_move = [0, 0, 0]
            final_move[action] = 1
            state, _, game_over, score = env_visual.play_step(final_move)
        score_result.append(score)  
    return score_result

def dqn_matrix_state(file_path, N_GAMES, visual=False, speed=0):
    model = keras.models.load_model(file_path)
    env_visual = enviroment.MatrixStateSnakeGame(visual=visual, speed=speed)
    score_result = []
    for _ in range(N_GAMES):
        state = env_visual.reset()
        state = env_visual.get_state()
        game_over = False
        while not game_over:
            action = np.argmax(model(state[np.newaxis])[0])
            final_move = [0,0,0,0]
            final_move[action] = 1
            state, _, game_over, score = env_visual.play_step(final_move)
        score_result.append(score)
    return score_result

Confronto tra differenti approcci ciascuno testato su 50 partite

In [16]:
score_result_t = tab_function(file_path="./Q_table/ExplorationFunction/890000step.npy",
             N_GAMES=50)
media_t = np.mean(score_result_t)
deviazione_std_t = np.std(score_result_t)
valore_max_t = np.max(score_result_t)
print(f"Media: {media_t}")
print(f"Deviazione standard: {deviazione_std_t}")
print(f"Valore massimo: {valore_max_t}")

Media: 29.94
Deviazione standard: 9.102549093523198
Valore massimo: 49


In [18]:
score_result_e = dqn_easy_state(file_path="./DQN_saved_model/easy_state/model_DDQN.keras",
             N_GAMES=50)
media_e = np.mean(score_result_e)
deviazione_std_e = np.std(score_result_e)
valore_max_e = np.max(score_result_e)
print(f"Media: {media_e}")
print(f"Deviazione standard: {deviazione_std_e}")
print(f"Valore massimo: {valore_max_e}")

Media: 26.56
Deviazione standard: 8.779886104044857
Valore massimo: 45


In [19]:
score_result_m = dqn_matrix_state(file_path="./DQN_saved_model/matrix_state/model_DDQN.keras",
             N_GAMES=50, visual=False, speed=10)
media_m = np.mean(score_result_m)
deviazione_std_m = np.std(score_result_m)
valore_max_m = np.max(score_result_m)
print(f"Media: {media_m}")
print(f"Deviazione standard: {deviazione_std_m}")
print(f"Valore massimo: {valore_max_m}")

Media: 37.28
Deviazione standard: 9.917741678426596
Valore massimo: 59


Test per verificare che il modello si accorga di morte imminente in caso di prosecuzione verso il frutto

In [3]:
def tab_test_one(file_path, speed=5):
    Q_table = np.load(file_path)
    env = enviroment.LinearStateSnakeGame(visual=True, speed=speed)
    state = env.reset()
    env.set_test_uno()
    state = env.get_state()
    game_over = False
    step_after_score=0
    while not game_over:
        state = env.get_state()
        row_Q_value = state_to_index(state)
        action = np.argmax(Q_table[row_Q_value])
        final_move = [0,0,0]
        final_move[action] = 1
        state, _, game_over, score = env.play_step(final_move)
        if score==1:
            if step_after_score>=10:
                print("Test succeded!")
                env.close_pygame()
                return
            else:
                step_after_score+=1
    print("Test failed!")       
    env.close_pygame()
    
def dqn_easy_test_one(file_path, speed=5):
    model = keras.models.load_model(file_path)
    env = enviroment.LinearStateSnakeGame(visual=True, speed=speed)
    state = env.reset()
    env.set_test_uno()
    state = env.get_state()
    game_over = False
    step_after_score=0
    while not game_over:
        action = np.argmax(model(state[np.newaxis])[0])
        final_move = [0,0,0]
        final_move[action] = 1
        state, _, game_over, score = env.play_step(final_move)
        if score==1:
            if step_after_score>=10:
                print("Test succeded!")
                env.close_pygame()
                return
            else:
                step_after_score+=1
    print("Test failed!")       
    env.close_pygame()

def dqn_matrix_test_one(file_path, speed=5):
    model = keras.models.load_model(file_path)
    env = enviroment.MatrixStateSnakeGame(visual=True, speed=speed)
    state = env.reset()
    env.set_test_uno()
    state = env.get_state()
    env.update_ui()
    game_over = False
    step_after_score=0
    while not game_over:
        action = np.argmax(model(state[np.newaxis])[0])
        final_move = [0,0,0,0]
        final_move[action] = 1
        state, _, game_over, score = env.play_step(final_move)
        if score>=1:
            if step_after_score>=10:
                print("Test succeded!")
                env.close_pygame()
                return
            else:
                step_after_score+=1
    print("Test failed!")       
    env.close_pygame()

In [23]:
tab_test_one("./Q_table/ExplorationFunction/890000step.npy", speed=5)

Test failed!


In [5]:
dqn_easy_test_one("./DQN_saved_model/easy_state/model_DDQN.keras", speed=5)

Test failed!


In [4]:
dqn_matrix_test_one("./DQN_saved_model/matrix_state/model_DDQN.keras", speed=5)

Test succeded!
